### Installation


In [2]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
%pip install --no-deps "xformers<0.0.29" "trl<0.9.0" peft accelerate bitsandbytes
%pip install --no-deps cut_cross_entropy unsloth_zoo
%pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

%pip install -U bitsandbytes
%pip install git+https://github.com/PanQiWei/AutoGPTQ.git@v0.4.2 




### Unsloth


In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",  # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",  # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",  # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",  # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",  # Gemma 2x faster!
    "unsloth/Llama-3.2-1B-bnb-4bit",  # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    "unsloth/Qwen2.5-7B",
    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit",  # NEW! Llama 3.3 70B!
]  # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B",  # or choose "unsloth/Llama-3.2-1B-Instruct"
    use_gradient_checkpointing=True,  # Tiết kiệm VRAM
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/nguyentrung/Documents/project/fine_tuning_model/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4060. Max memory: 7.739 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth 2025.2.12 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Load Dataset


In [5]:
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
from datasets import load_dataset, concatenate_datasets

system_message ="""
    Bạn là chunGPT được phát triển bởi đại ca Vương Nguyên Trung.
    Your role as an assistant involves thoroughly exploring questions through a systematic long thinking process before providing the final precise and accurate solutions. This requires engaging in a comprehensive cycle of analysis, summarizing, exploration, reassessment, reflection, backtracing, and iteration to develop well-considered thinking process. Please structure your response into two main sections: Thought and Solution. In the Thought section, detail your reasoning process using the specified format:
    <reasoning>
        {reasoning with steps separated with '\n\n'}
    </reasoning>
    <answer>
        {final formatted, precise, and clear solution}
    </answer>
"""
# First load tokenizer and dataset
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
    # chat_template="chatml",
    # system_message=system_message,
)

dataset = load_dataset("1TuanPham/Vietnamese-OpenO1-SFT", split="vi")
# en_dataset = load_dataset("1TuanPham/Vietnamese-OpenO1-SFT", split="en")

# Kết hợp và xử lý định dạng
# dataset = concatenate_datasets([vi_dataset, en_dataset])
# (Tùy chọn) Lọc các cột không cần thiết nếu muốn
dataset = dataset.remove_columns(["qas_id"])  # Giữ lại nếu cần ID


def add_conversations_column(examples):
    conversations = [
        [
            {
                "role": "system",
                "content": system_message,
            },
            {
                "role": "user",
                "content": f"{instr}",
            },
            {
                "role": "assistant",
                "content": f"<reasoning>{cot}</reasoning>\n\n<answer>{out}</answer>",
            },
        ]
        for instr, cot, out in zip(
            examples["instruction"], examples["cot"], examples["output"]
        )
    ]
    return {"conversations": conversations}


dataset = dataset.map(add_conversations_column, batched=True)
dataset = standardize_sharegpt(dataset)


def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = []
    for convo in convos:
        # Thêm cờ truncation và max_length
        text = tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False,
            truncation=True,
            max_length=2048,
        )
        texts.append(text)
    return {"text": texts}


dataset = dataset.map(formatting_prompts_func, batched=True)

In [6]:
# Kiểm tra template mẫu
test_convo = [
    {"role": "system", "content": "Test system"},
    {"role": "user", "content": "Test instruction"},
    {"role": "assistant", "content": "Test response"},
]
print(tokenizer.apply_chat_template(test_convo, tokenize=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

Test system<|eot_id|><|start_header_id|>user<|end_header_id|>

Test instruction<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Test response<|eot_id|>


In [7]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n\n    Bạn là chunGPT được phát triển bởi đại ca Vương Nguyên Trung.\n    Your role as an assistant involves thoroughly exploring questions through a systematic long thinking process before providing the final precise and accurate solutions. This requires engaging in a comprehensive cycle of analysis, summarizing, exploration, reassessment, reflection, backtracing, and iteration to develop well-considered thinking process. Please structure your response into two main sections: Thought and Solution. In the Thought section, detail your reasoning process using the specified format:\n    <reasoning>\n        {reasoning with steps separated with \'\n\n\'}\n    </reasoning>\n    <answer>\n        {final formatted, precise, and clear solution}\n    </answer>\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nViết cho tôi một quy trình làm việc AWS step function

In [ ]:
dataset[5]["conversations"]

[{'content': "\n    Bạn là chunGPT được phát triển bởi đại ca Vương Nguyên Trung.\n    Your role as an assistant involves thoroughly exploring questions through a systematic long thinking process before providing the final precise and accurate solutions. This requires engaging in a comprehensive cycle of analysis, summarizing, exploration, reassessment, reflection, backtracing, and iteration to develop well-considered thinking process. Please structure your response into two main sections: Thought and Solution. In the Thought section, detail your reasoning process using the specified format:\n    <reasoning>\n        {reasoning with steps separated with '\n\n'}\n    </reasoning>\n    <answer>\n        {final formatted, precise, and clear solution}\n    </answer>\n",
  'role': 'system'},
 {'content': 'Viết cho tôi một quy trình làm việc AWS step function ở định dạng json, quy trình này đầu tiên chuyển đổi dữ liệu JSON bằng cách sử dụng một mẫu ánh xạ trước khi chuyển nó đến một hàm lamb

<a name="Train"></a>

### Train the model

Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!


In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=3,
        warmup_steps=100,
        num_train_epochs=2,  # Set this for 1 full training run.
        # max_steps=300,
        # save_steps=250,
        learning_rate=5e-6,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=50,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="outputs",
        report_to="none",  # Use this for WandB etc
    ),
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.


In [10]:
from unsloth.chat_templates import train_on_responses_only

if True:
    trainer = train_on_responses_only(
        trainer,
        instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
        response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
    )

if False:
    trainer = train_on_responses_only(
        trainer,
        instruction_part="<|im_start|>user\n",  # Ví dụ cho ChatML
        response_part="<|im_start|>assistant\n",  # Ví dụ cho ChatML
        # Hoặc cho Llama 3:
        # instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
        # response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
    )

We verify masking is actually done:


In [11]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n\n    Bạn là chunGPT được phát triển bởi đại ca Vương Nguyên Trung.\n    Your role as an assistant involves thoroughly exploring questions through a systematic long thinking process before providing the final precise and accurate solutions. This requires engaging in a comprehensive cycle of analysis, summarizing, exploration, reassessment, reflection, backtracing, and iteration to develop well-considered thinking process. Please structure your response into two main sections: Thought and Solution. In the Thought section, detail your reasoning process using the specified format:\n    <reasoning>\n        {reasoning with steps separated with \'\n\n\'}\n    </reasoning>\n    <answer>\n        {final formatted, precise, and clear solution}\n    </answer>\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nViết cho tôi một quy trình làm việc 

In [12]:
space = tokenizer(" ", add_special_tokens=False).input_ids[0]
tokenizer.decode(
    [space if x == -100 else x for x in trainer.train_dataset[6]["labels"]]
)

'                                                                                                                                                                                                                                                 \n\n<reasoning>Được thôi, tôi cần tìm cách sửa đổi code hiện có để quản lý mảng các hành tinh trong Hệ Mặt Trời của chúng ta để thêm một hành tinh mới. Hành tinh mới phải tuân theo quy ước đặt tên của thần thoại cổ điển và phải được định vị sao cho nó không gần Mặt Trời hơn Sao Thủy hoặc xa hơn Sao Diêm Vương. Chúng ta hãy chia nhỏ vấn đề này từng bước.\n\nĐầu tiên, tôi nên xem xét cấu trúc hiện tại của mảng hành tinh. Nó có khả năng là một mảng các đối tượng, trong đó mỗi đối tượng đại diện cho một hành tinh với các thuộc tính như tên, khoảng cách từ Mặt Trời và có thể các thuộc tính khác. Để thêm một hành tinh mới, tôi sẽ cần tạo một đối tượng mới với các thuộc tính này.\n\nTiếp theo, quy ước đặt tên yêu cầu tên của hành tinh mới phải bắt nguồn t

We can see the System and Instruction prompts are successfully masked!


In [13]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4060. Max memory = 7.739 GB.
2.301 GB of memory reserved.


In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,372 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 3
\        /    Total batch size = 6 | Total steps = 1,790
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss


KeyboardInterrupt: 

In [15]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

NameError: name 'trainer_stats' is not defined

<a name="Inference"></a>

### Inference


In [16]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Tiếp tục chuỗi fibonacci: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,  # Must add for generation
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs, max_new_tokens=128, use_cache=True, temperature=1.5, min_p=0.1
)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nTiếp tục chuỗi fibonacci: 1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nTạo hàm fibonacci lấy từ biến fibonacci có giá trị ban đầu 0,1:\n\nTiếp tục chuỗi fibonacci: 1, 1, 2, 3, 5, 8, 13, 21, 34,...\n\nTiếp tục chuỗi fibonacci: 1, 1, 2, 3, 5, 8, 13, 21,...\n\nTiếp tục chuỗi fibonacci: 1, 1, 2, 3, 5, 8,...\n\nTiếp tục chuỗi fibonacci: 1, 1, 2, 3,...\n\nTi']

You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!


In [ ]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Tiếp tục chuỗi Fibonacci: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,  # Must add for generation
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    input_ids=inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.5,
    min_p=0.1,
)

<a name="Save"></a>

### Saving, loading finetuned models

To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!


In [17]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:


In [ ]:
if True:
    from unsloth import FastLanguageModel

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

messages = [
    {"role": "user", "content": system_message},
    {"role": "user", "content": "Tiếp tục chuỗi fibonacci: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,  # Must add for generation
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    input_ids=inputs,
    streamer=text_streamer,
    max_new_tokens=4000,
    use_cache=True,
    temperature=0.5,
    min_p=0.1,
)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.


In [ ]:
if True:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer

    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

# Hoặc dùng cách dưới


In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

# Load model và tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    "load_models",
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained("lora_model")

# Merge LoRA vào base model
merged_model = model.merge_and_unload()  # ← Bước quan trọng!

# Lưu model đã merge
merged_model.save_pretrained(
    "output/merged_model",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=False,
)
tokenizer.save_pretrained("output/merged_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.


In [ ]:
# Merge to 16bit
if True:
    model.save_pretrained(
        save_directory="ChunThinker/lora_model_vllm",
        tokenizer=tokenizer,  # Lưu tokenizer riêng
        save_method="merged_16bit",  # Merge adapter vào base model
        safe_serialization=True,  # Bắt buộc để tránh lỗi
    )
if False:
    model.push_to_hub_merged(
        "hf/model", tokenizer, save_method="merged_16bit", token=""
    )

# Merge to 4bit
if False:
    model.save_pretrained_merged(
        "model",
        tokenizer,
        save_method="merged_4bit",
    )
if False:
    model.push_to_hub_merged("hf/model", tokenizer, save_method="merged_4bit", token="")

# Just LoRA adapters
if False:
    model.save_pretrained_merged(
        "model",
        tokenizer,
        save_method="lora",
    )
if False:
    model.push_to_hub_merged("hf/model", tokenizer, save_method="lora", token="")

### GGUF / llama.cpp Conversion

To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):

- `q8_0` - Fast conversion. High resource use, but generally acceptable.
- `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
- `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)


In [ ]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf(
        "model",
        tokenizer,
    )
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token="")

# Save to 16bit GGUF
if True:
    model.save_pretrained(
        "output/model/gguf", tokenizer, quantization_method="f16"
    )
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method="f16", token="")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method="q4_k_m")
if False:
    model.push_to_hub_gguf(
        "hf/model", tokenizer, quantization_method="q4_k_m", token=""
    )

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model",  # Change hf to your username!
        tokenizer,
        quantization_method=[
            "q4_k_m",
            "q8_0",
            "q5_k_m",
        ],
        token="",  # Get a token at https://huggingface.co/settings/tokens
    )